In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.model_selection import GridSearchCV
np.random.seed(0)

In [2]:

def scaler(scaler, data, test=None):
    scaler.fit(data)  # Apply transform to both the training set and the test set.
    train_scale = scaler.transform(data)
    if test is not None:
        test_scale = scaler.fit_transform(test)

    return train_scale, test_scale, scaler

def train_model(classifier, X_tr, y_tr, X_te, y_te):
    print('start training...')
    classifier.fit(X_tr, y_tr)
    print('evaluation...')
    y_p = classifier.predict(X_te)
    score = evaluate(y_te, y_p)
    print(f'score is {score}')
    return classifier, score

def evaluate(y_true, y_pred):
    return f1_score(y_true, y_pred, average="macro")

In [3]:
data = pd.read_table('data/dev.tsv')
test = pd.read_table('data/eval.tsv')

In [4]:
df = data.copy()
eval = test.copy()

In [59]:
from scipy import stats
def happy_sad(x):
    if x>df['valence'].mean():
        return 'happy'
    else:
        return 'sad'
df['boringness'] = df['loudness'] + df['tempo'] + (df['energy']*100) + (df['danceability']*100)
df['valence_happy_sad'] = df['valence'].apply(lambda x: happy_sad(x))
df['loudness_plus_60'] = df['loudness'].apply(lambda x: x+60)
df['loudness_pos'] = df['loudness'].apply(lambda x: -1*x)
df['loudness_pos'] = np.sqrt(df['loudness_pos'])
df['boringness_plus_60'] = df['boringness'].apply(lambda x: x+60)
df['duration_ms_box_cox_trans'] = stats.boxcox(df['duration_ms'])[0]
df['acousticness_sqrt_trans'] = np.sqrt(df['acousticness'])
df['liveness_sqrt_trans'] = np.sqrt(df['liveness'])
df['popularity_sqrt_trans'] = np.sqrt(df['popularity'])
df['speechiness_sqrt_trans'] = np.sqrt(df['speechiness'])

/home/omid/anaconda3/envs/ml/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [107]:
df = df.fillna(value=0)
df.describe().T

,count,mean,std,min,25%,50%,75%,max
valence,136522.0,0.528540,0.263282,0.000000,0.317000,0.540000,0.747000,1.000000e+00
year,136522.0,1976.796048,25.921025,1921.000000,1956.000000,1977.000000,1999.000000,2.020000e+03
acousticness,136522.0,0.502159,0.376037,0.000000,0.102000,0.516000,0.893000,9.960000e-01
danceability,136522.0,0.537497,0.175972,0.000000,0.415000,0.548000,0.668000,9.880000e-01
duration_ms,136522.0,231061.453041,127800.577241,5991.000000,169933.000000,207493.000000,262267.000000,5.403500e+06
energy,136522.0,0.482298,0.267538,0.000000,0.255000,0.471000,0.703000,1.000000e+00
explicit,136522.0,0.084411,0.278005,0.000000,0.000000,0.000000,0.000000,1.000000e+00
instrumentalness,136522.0,0.167172,0.313448,0.000000,0.000000,0.000218,0.103000,1.000000e+00
key,136522.0,5.193200,3.513427,0.000000,2.000000,5.000000,8.000000,1.100000e+01
liveness,136522.0,0.205708,0.174404,0.000000,0.098900,0.136000,0.261000,1.000000e+00


In [171]:
col = [
    'valence',
        'year',
        # 'acousticness',
        # 'artists',
         'danceability',
       'duration_ms',
        'energy',
        'explicit',
        # 'id',
        'instrumentalness',
        'key',
       'liveness',
        # 'loudness',
        # 'popularity',
        # 'speechiness',
        'tempo',
       #  'mode',
       # 'loudness_plus_60',
        'loudness_pos',
         # 'boringness',
       #  'valence_happy_sad',
       # 'boringness_plus_60',
       #  'duration_ms_box_cox_trans',
       'acousticness_sqrt_trans',
       #  'liveness_sqrt_trans',
       # 'popularity_sqrt_trans',
        'speechiness_sqrt_trans'
        ]
X = df[col]
y = df['mode']

In [180]:
from imblearn.under_sampling import RandomUnderSampler,NearMiss
ous = RandomUnderSampler(random_state=42)
nm = NearMiss()
X,y = nm.fit_sample(X,y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0)

In [213]:
from sklearn.preprocessing import *
X_train_scal, X_test_scal, x_scaler = scaler(RobustScaler(), X_train, X_test)

In [214]:
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

# clf = SVC(random_state=42)
# clf = GaussianNB()
# clf = MultinomialNB(alpha=10/100)
# clf = BernoulliNB(alpha=10/100)
# clf = LogisticRegression(penalty='l2',max_iter=10000,random_state=0)
clf = RandomForestClassifier(n_estimators=1000,n_jobs=-1, random_state=0)
# clf = SGDClassifier()
# clf = KNeighborsClassifier(n_neighbors = 10)
# clf = MLPClassifier(early_stopping=True,random_state=1,max_iter=500, verbose=True)
# clf = LGBMClassifier()
# clf= XGBClassifier()

score = train_model(clf,X_train_scal,y_train,X_test_scal,y_test)
print(f' score {score}')
# scores = cross_val_score(clf, X, y, cv=5,scoring='f1_macro')
# print(f'mean score {np.mean(scores)}, max score {np.max(scores)}, min score {np.min(scores)}')

start training...
evaluation...
score is 0.6567935976769663
 score (RandomForestClassifier(n_estimators=1000, n_jobs=-1, random_state=0), 0.6567935976769663)


In [215]:
print(f' score {score}')

 score (RandomForestClassifier(n_estimators=1000, n_jobs=-1, random_state=0), 0.6567935976769663)


In [141]:
# from sklearn.neighbors import KNeighborsClassifier
#
# score = []
# for i in range(2,100,2):
#     knn = KNeighborsClassifier(n_neighbors = i)
#     knn.fit(X_train, y_train)
#     y_pred = knn.predict(X_test)
#     sco = evaluate(y_test, y_pred)
#     score.append(sco)
#     print(f'{i} {sco:.4f}')

In [157]:
def WriteOnFile(name, y_eval):
    f = open(name, "w")
    f.write("Id,Predicted\n")
    for index, i in enumerate(y_eval):
        f.write(f"{index},{i}\n")
    f.close

eval['boringness'] = eval['loudness'] + eval['tempo'] + (eval['energy']*100) + (eval['danceability']*100)
eval['valence_happy_sad'] = eval['valence'].apply(lambda x: happy_sad(x))
eval['loudness_plus_60'] = eval['loudness'].apply(lambda x: x+60)
eval['loudness_pos'] = eval['loudness'].apply(lambda x: -1*x)
eval['loudness_pos'] = np.sqrt(eval['loudness_pos'])
eval['boringness_plus_60'] = eval['boringness'].apply(lambda x: x+60)
eval['duration_ms_box_cox_trans'] = stats.boxcox(eval['duration_ms'])[0]
eval['acousticness_sqrt_trans'] = np.sqrt(eval['acousticness'])
eval['liveness_sqrt_trans'] = np.sqrt(eval['liveness'])
eval['popularity_sqrt_trans'] = np.sqrt(eval['popularity'])
eval['speechiness_sqrt_trans'] = np.sqrt(eval['speechiness'])
eval = eval.fillna(value=0)

/home/omid/anaconda3/envs/ml/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [216]:
test = eval[col]
test_scal = x_scaler.transform(test)
y_pred = clf.predict(test_scal)
WriteOnFile('submission.csv',y_pred)